In [1]:
from model import *
from data import *

/anaconda/envs/py35/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Train your Unet with data
data is in folder /data/your_image_folder/, it is a binary classification task.

```bash
data/your_image_folder
├── train
│   ├── image
│   │   ├── 0.png
│   │   ├── 1.png
│   ├── label
│   │   ├── 0.png
│   │   ├── 1.png
├── test
│   ├── image
│   │   ├── 0.png
│   │   ├── 1.png
│   ├── label
│   │   ├── 0.png
│___│___├── 1.png

```

### Train with data generator

In [ ]:
# transform the training data to contrast gray image
import cv2
import matplotlib.pyplot as plt
image = cv2.imread("data/image_folder/train/image/train.JPG", cv2.IMREAD_GRAYSCALE)
gray = 255 - image
cv2.imwrite("data/image_folder/train/image/0.png", gray)

In [ ]:
# train UNET with transfer learning
data_gen_args = dict(rotation_range=0.2,
                    width_shift_range=0.05,
                    height_shift_range=0.05,
                    shear_range=0.05,
                    zoom_range=0.05,
                    horizontal_flip=True,
                    fill_mode='nearest')
myGene = trainGenerator(2,'data/image_folder/train','image','label',data_gen_args,save_to_dir = None)
model = unet('unet_membrane.hdf5') # transfer learning or you can remove the pre-trained model
model_checkpoint = ModelCheckpoint('unet_transfer_learning_model.hdf5', monitor='loss',verbose=1, save_best_only=True)
model.fit_generator(myGene,steps_per_epoch=500,epochs=2,callbacks=[model_checkpoint])

### test your model and save predicted results

In [ ]:
number_of_test_images = 1
testGene = testGenerator("data/image_folder/test", num_image = number_of_test_images)
model = unet()
model.load_weights("unet_transfer_learning_model.hdf5")
results = model.predict_generator(testGene, number_of_test_images, verbose=1)

In [ ]:
saveResult("data/image_folder/test/result",results)

# watershed

In [6]:
from skimage import io, color
from skimage import filters
from scipy import ndimage

from skimage.morphology import watershed
from skimage.feature import peak_local_max
from skimage.measure import regionprops, label

import cv2
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
bin_threshold = 10
background_threshold = 30
foreground _threshold = 200

for i in range(0, numter_of_test_images):
    imresult = results[i,:,:,0]
    ret, thresh = cv2.threshold(255 - imresult*255, bin_threshold, 255,cv2.THRESH_BINARY_INV) # transform the unet result to binary image
    thresh=thresh/255 # normalize to 1
    
    edges = ndimage.sobel(thresh) # filter with sobel
    markers = np.zeros_like(thresh)
    foreground, background = 1, 2
    
    # set background and foreground with low and high threshold
    markers[thresh < background_threshold / 255] = background
    markers[thresh > foreground_threshold / 255] = foreground
    
    ws = watershed(edges, markers)
    seg = ndimage.label(ws == foreground)[0]
    
    # get centroids and areas
    regions = regionprops(seg)
    regions = [r for r in regions if r.area > 10]
    print('Number of centroids for image '+str(i) + ': ', len(regions))
    centroids = [r.centroid for r in regions if r.area > 10]
    x = [i[0] for i in centroids]
    y = [i[1] for i in centroids]
    
    #plot centroid on original image
    image = cv2.imread(os.path.join("data/image_folder/test", str(i) + ".png"))
    im_shape = image.shape
    x_scale =  [x_single*im_shape[0]/256 for x_single in x]
    y_scale =  [y_single*im_shape[1]/256 for y_single in y]
    image_square = cv2.resize(image, (256,256))
    image_square[markers==0] = (255, 0 , 0)
    image_square = cv2.resize(image_square,im_shape[0:2])
    
    
    plt.figure()
    plt.imshow(image, cmap='gray')
    plt.scatter(y_scale,x_scale, c = 'r',marker = 'o')
    plt.title("Number " + str(i))
    plt.savefig(os.path.join("data/image_folder/test/result", str(i) + "_dots.png"))
